In [1]:
# Imports
import pickle
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../../../')

from src import customFunc as cf
from src import ansatzs as anz

In [2]:
# Hamiltonians
with open(f"data/hamiltonian{0.719}.pkl", "rb") as f:
    hamiltonian = pickle.load(f)
# Repulsion energies
with open(f"data/nuclear_repulsion{0.719}.pkl", "rb") as f:
    nuclear_repulsion = pickle.load(f)

In [ ]:
ansatz=anz.optimize_qaoa(hamiltonian)

CircuitError: 'Expression cannot bind non-numeric values ({ParameterVectorElement(γ[0]): ParameterVector(name=γ, length=9)})'